In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import mean_squared_error, r2_score
import json
df=pd.read_csv("/home/ubuntu/exam_dvc/examen-dvc/data/raw/raw.csv")
df.head()


,date,ave_flot_air_flow,ave_flot_level,iron_feed,starch_flow,amina_flow,ore_pulp_flow,ore_pulp_pH,ore_pulp_density,silica_concentrate
0,2017-04-24 00:00:00,300.263166,383.982443,55.17,1979.589150,599.676489,400.017222,9.774028,1.753206,4.360000
1,2017-04-24 01:00:00,299.782402,386.049069,55.17,1758.466329,600.043100,400.484528,9.539246,1.754190,3.290000
2,2017-04-24 02:00:00,299.750052,385.250935,55.17,2379.752428,599.948406,400.325617,9.434227,1.756873,4.900000
3,2017-04-24 03:00:00,299.997522,389.635519,55.17,2287.130046,599.580383,399.801506,9.725607,1.727125,4.860153
4,2017-04-24 04:00:00,300.005220,387.810807,55.17,2291.789167,599.871217,399.567333,9.845198,1.633063,4.780898


In [ ]:
df.describe()

,ave_flot_air_flow,ave_flot_level,iron_feed,starch_flow,amina_flow,ore_pulp_flow,ore_pulp_pH,ore_pulp_density,silica_concentrate
count,1817.000000,1817.000000,1817.000000,1817.000000,1817.000000,1817.000000,1817.000000,1817.000000,1817.000000
mean,296.036027,457.718993,57.594662,3106.797900,502.075958,400.015391,9.895610,1.681849,2.161673
std,6.342455,66.963782,5.753694,831.195329,65.957732,0.865649,0.298358,0.049554,1.047229
min,282.466012,306.748910,43.370000,401.628723,301.264021,396.331445,8.809455,1.546127,0.850000
25%,291.643054,408.780472,52.670000,2554.580328,454.769861,399.508139,9.705173,1.653314,1.370000
50%,299.859804,421.505544,57.380000,3144.695311,509.368744,399.976989,9.914017,1.690384,1.820000
75%,299.961731,500.251061,64.030000,3648.292550,553.007194,400.482357,10.103874,1.716987,2.670000
max,301.338649,767.639752,65.780000,5556.094067,715.748025,403.177721,10.758195,1.797635,5.330000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1817 entries, 0 to 1816
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                1817 non-null   object 
 1   ave_flot_air_flow   1817 non-null   float64
 2   ave_flot_level      1817 non-null   float64
 3   iron_feed           1817 non-null   float64
 4   starch_flow         1817 non-null   float64
 5   amina_flow          1817 non-null   float64
 6   ore_pulp_flow       1817 non-null   float64
 7   ore_pulp_pH         1817 non-null   float64
 8   ore_pulp_density    1817 non-null   float64
 9   silica_concentrate  1817 non-null   float64
dtypes: float64(9), object(1)
memory usage: 142.1+ KB


In [2]:
X = df.drop(['date','silica_concentrate'],axis=1)
y = df['silica_concentrate']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:

model = Ridge()
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  #
    'solver': ['auto', 'svd', 'cholesky']
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.1, 1.0, 10.0],
                         'solver': ['auto', 'svd', 'cholesky']},
             scoring='r2')

In [5]:
joblib.dump(grid_search.best_params_, '/home/ubuntu/exam_dvc/examen-dvc/src/models/best_params.pkl')

['/home/ubuntu/examen-dvc/src/models/best_params.pkl']

In [6]:
best_params = joblib.load('/home/ubuntu/exam_dvc/examen-dvc/src/models/best_params.pkl')

model = Ridge(**best_params)
model.fit(X_train, y_train.values.ravel())

joblib.dump(model, '/home/ubuntu/exam_dvc/examen-dvc/src/models/model.pkl')

['/home/ubuntu/examen-dvc/src/models/model.pkl']

In [7]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

metrics = {'mse': mse, 'r2': r2}
with open('/home/ubuntu/exam_dvc/examen-dvc/metrics/scores.json', 'w') as f:
    json.dump(metrics, f)

pd.DataFrame(y_pred, columns=['predictions']).to_csv('/home/ubuntu/exam_dvc/examen-dvc/data/predictions.csv', index=False)

In [8]:
from sklearn.metrics import mean_squared_error
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mse = mean_squared_error(y_true, y_pred)
print(mse)


0.375
